# RSDC
Structure:
1. Simulate data
2. Original Estimation Functions
3. Expectation Functions
4. Uniting functions



In [1]:
import numpy as np
import time
from tqdm import tqdm

## Simulation
***


In [3]:
def simulate_data(K,T, parameters, transition_matrix, cholesky):
    # Initialize arrays to store the processes and variances
    processes = np.zeros((K, T))
    variances = np.zeros((K, T))
    states = np.zeros((T))
    innovations = np.zeros((K, T))
    
    # Initial variance (can be set equal to the long-term variance)
    variances[:, 0] = parameters[:, 0] / (1 - parameters[:, 1] - parameters[:, 2])
    states[0] = 0
    # Simulate the GARCH processes
    for t in range(1, T):
        state = int(states[t-1])
        current_state = np.random.choice(a=[0,1], p=transition_matrix[state])
        states[t] = current_state
        
        # Simulate standard normal innovations
        z = np.random.normal(0, 1, K)
        # Apply the Cholesky decomposition to introduce correlation
        correlated_z = cholesky[current_state,:,:] @ z
        # Update the variance
        variances[:, t] = parameters[:, 0] + parameters[:, 1] * (processes[:, t-1]**2) + parameters[:, 2] * variances[:, t-1]
        # Calculate the process values
        processes[:, t] = np.sqrt(variances[:, t]) * correlated_z[:]
    return processes, states, variances, innovations

def cholesky_form(matrix, N, K):
    cholesky = np.zeros((N, K, K))
    for n in range(N):
        cholesky[n,:,:] = np.linalg.cholesky(true_correlation_matrix[n,:,:])

    return cholesky
import numpy as np
def vectorized_simulate_data(K, T, parameters, transition_matrix, cholesky):
    # Pre-allocate arrays
    processes = np.zeros((K, T))
    variances = np.zeros((K, T))
    states = np.zeros(T, dtype=int)
    innovations = np.random.normal(0, 1, (K, T))  # Pre-generate all innovations
    
    # Initial variance
    variances[:, 0] = parameters[:, 0] / (1 - parameters[:, 1] - parameters[:, 2])
    
    # Simulate states with a Markov chain
    # This part is inherently sequential but let's try to minimize the loop's impact.
    for t in range(1, T):
        states[t] = np.random.choice([0, 1], p=transition_matrix[states[t-1]])
    
    # Vectorize the correlation application using pre-simulated innovations
    # Note: This approach changes the structure slightly, as we need to correlate all innovations first and then select based on state
    correlated_innovations = np.zeros((K, T))
    for state in range(transition_matrix.shape[0]):
        # Find indices where this state occurs
        state_indices = np.where(states == state)[0]
        if len(state_indices) > 0:
            for i in state_indices:
                correlated_innovations[:, i] = cholesky[state] @ innovations[:, i]
    
    # Update variances and processes in a vectorized way
    for t in range(1, T):
        variances[:, t] = parameters[:, 0] + parameters[:, 1] * (processes[:, t-1]**2) + parameters[:, 2] * variances[:, t-1]
        processes[:, t] = np.sqrt(variances[:, t]) * correlated_innovations[:, t]
    
    return processes, states, variances, innovations




In [4]:
univariate_parameters = np.array([[0.05, 0.15, 0.75],
                      [0.1, 0.25, 0.6] ])#,
                      # [0.15, 0.15, 0.8],
                      # [0.2, 0.3, 0.65]])
true_correlation_matrix = np.array([[[1, 0.7],[0.7,1]],[[1, -0.6],[-0.6, 1]]])
# true_correlation_matrix = np.array([
#     # High correlations
#     [[1, 0.8, 0.7, 0.6],
#      [0.8, 1, 0.65, 0.55],
#      [0.7, 0.65, 1, 0.5],
#      [0.6, 0.55, 0.5, 1]],
    
#     # Low correlations
#     [[1, 0.1, 0.15, 0.05],
#      [0.1, 1, 0.2, 0.1],
#      [0.15, 0.2, 1, 0.12],
#      [0.05, 0.1, 0.12, 1]],
    
#     # Negative correlations
#     [[1, -0.5, -0.4, -0.3],
#      [-0.5, 1, -0.2, -0.1],
#      [-0.4, -0.2, 1, -0.25],
#      [-0.3, -0.1, -0.25, 1]],
    
#     # In between
#     [[1, 0.4, -0.3, 0.2],
#      [0.4, 1, 0.25, -0.2],
#      [-0.3, 0.25, 1, 0.15],
#      [0.2, -0.2, 0.15, 1]]
# ])
diagonal = 0.99
N = 2
K = 2
true_transition_matrix = diagonal * np.eye(N) + (1-diagonal) * (np.ones((N,N)) - np.eye(N,N)) / (N - 1)

cholesky = cholesky_form(true_correlation_matrix, N, K)

# Number of observations
T = 100
# start_time = time.time()

# data, states, variances, innovations = simulate_data(K,T, univariate_parameters, true_transition_matrix, cholesky)

# # End timing
# end_time = time.time()

# # Calculate elapsed time
# elapsed_time = end_time - start_time

# # Print elapsed time with 5 digits after the decimal point
# print(f"Elapsed time: {elapsed_time:.5f} seconds")



In [5]:
start_time = time.time()

data, states, variances, innovations = vectorized_simulate_data(K,T, univariate_parameters, true_transition_matrix, cholesky)

# End timing
end_time = time.time()

# Calculate elapsed time
elapsed_time = end_time - start_time

# Print elapsed time with 5 digits after the decimal point
print(f"Elapsed time: {elapsed_time:.5f} seconds")


Elapsed time: 0.00668 seconds


## Original Estimation Functions



In [6]:

def calculate_standard_deviations(data, univariate_parameters):
    K,T = data.shape
    sigmas = np.zeros((K,T))
    sigmas[:,0] = np.var(data, axis=1)
    for t in range(1, T):
        sigmas[:,t] = univariate_parameters[:,0] + univariate_parameters[:, 1] * data[:, t-1]**2 + univariate_parameters[:, 2] * sigmas[:, t-1]    
    return np.sqrt(sigmas)

def calculate_residuals(data, sigma):
    return data / sigma



def form_states(states, N, T):
    # Ensure states are integers within the correct range
    states = np.clip(states.astype(int), 0, N-1)
    
    # Initialize the state array with zeros
    state_array = np.zeros((N, T))
    
    # Mark ones for active states
    for t, state in enumerate(states):
        state_array[state, t] = 1
    
    return state_array

import numpy as np

def generate_initial_correlation_matrices(N, K):
    # Create an array to hold the correlation matrices
    correlation_matrices = np.zeros((N, K, K))
    
    # Define the pattern of off-diagonal values based on N
    off_diagonal_values = np.linspace(-0.1, 0.1 * N, N)
    
    for i in range(N):
        # Fill the diagonal with 1s
        np.fill_diagonal(correlation_matrices[i], 1)
        
        # Fill the off-diagonal elements with the specified pattern
        np.fill_diagonal(correlation_matrices[i, :, 1:], off_diagonal_values[i])
        np.fill_diagonal(correlation_matrices[i, 1:, :], off_diagonal_values[i])
        
        # Ensure the matrix is symmetric
        correlation_matrices[i] = (correlation_matrices[i] + correlation_matrices[i].T) / 2

    for i in range(N):
        correlation_matrices[i] = cholesky_scale(correlation_matrices[i])
    
    return correlation_matrices

    
def cholesky_scale(matrix):
    K, E = matrix.shape
    P = np.linalg.cholesky(matrix)
    for j in range(K):
        sum = np.sum(P[j, :j] ** 2)
        P[j,j] = np.sqrt(1-sum) if 1 - sum > 0 else 0
    scaled = np.dot(P, P.T)
    return scaled
correlation_matrices = generate_initial_correlation_matrices(N, K)
correlation_matrices


array([[[ 1. , -0.1],
        [-0.1,  1. ]],

       [[ 1. ,  0.2],
        [ 0.2,  1. ]]])

## Expectation Functions
***
**Initial State**

**Calculate Densities**

**Forward Pass**

**Backward Pass**

**Smoothed Probabilities**

**Estimate Transition Probabilities**

In [7]:
def initial_state(N):
    # Initial_state = 1/N
    initial_states = np.ones(N) / N
    return initial_states
    
def get_densities(N, K, T, standard_deviations, correlation_matrices, residuals):
    # Calculate Inverse and Determinant of the correlation matrix R
    det_R = np.zeros(N)
    inv_R = np.zeros((N,K,K))
    for n in range(N):
        det_R[n] = np.linalg.det(correlation_matrices[n,:,:])
        inv_R[n,:,:] = np.linalg.inv(correlation_matrices[n,:,:])
    # Use log for the determinant part to avoid overflow
    log_determinants = np.sum(np.log(standard_deviations), axis=0)
    
    # Initial log densities
    initial_log_densities = np.ones((N, T)) * K * np.log(2 * np.pi)
    
    # Combine initial log densities and log determinants
    log_densities = initial_log_densities + log_determinants[np.newaxis, :]
    
    # Adjust log_densities for broadcasting
    det_R_adjusted = det_R[:, np.newaxis]
    
    # Calculate z_t @ R_inv @ z_t' in a numerically stable manner
    intermediate_result = np.einsum('nkl,lt->nkt', inv_R, residuals)
    final_result = np.einsum('nkt,kt->nt', intermediate_result, residuals)
    
    # Combine to update log densities
    log_densities += -0.5 * (det_R_adjusted + final_result)
    
    # Convert log densities to densities safely
    max_log_densities = np.max(log_densities, axis=0, keepdims=True)
    densities = np.exp(log_densities - max_log_densities)
    return det_R, inv_R, densities

def forward_pass(N, T, initial_states, densities, transition_matrix):
    # Initialize forward probabilities & Scale factors
    forward_probabilities = np.zeros((N, T))
    scale_factors = np.zeros(T)

    # Set observation 0
    forward_probabilities[:, 0] = initial_states * densities[:,0]
    scale_factors[0] = 1.0 / np.sum(forward_probabilities[:, 0], axis = 0)
    forward_probabilities[:,0] *= scale_factors[0, np.newaxis]

    # Loop through all T
    for t in range(1, T):
        forward_probabilities[:, t] = np.dot(forward_probabilities[:,t-1], transition_matrix) * densities[:,t]
        scale_factors[t] = 1.0 / np.sum(forward_probabilities[:, t], axis = 0)
        forward_probabilities[:,t] *= scale_factors[t, np.newaxis]
    # Return Scales and forward probabilities
    return forward_probabilities, scale_factors

def backward_pass(N, T, scale_factors, transition_matrix, densities):
    # Initialize Backward probabilitiy array
    backward_probabilities = np.zeros((N, T))

    # set observation 0
    backward_probabilities[:, T-1] = 1.0 * scale_factors[T-1, np.newaxis]

    # Loop from T-2 to -1
    for t in range(T-2, -1, -1):
        backward_probabilities[:,t] = np.dot(transition_matrix, (densities[:,t+1] * backward_probabilities[:, t+1]))
        
        # Scale to prevent underflow
        backward_probabilities[:,t] *= scale_factors[t, np.newaxis]
    return backward_probabilities

def calculate_smoothed_probabilities(forward_probabilities, backward_probabilities, transition_matrix, densities):
    # Smoothed State probabilities
    numerator = forward_probabilities * backward_probabilities
    denominator = numerator.sum(axis=0, keepdims=True)
    u_hat = numerator / denominator

    # Initial state probabilities
    delta = u_hat[:,0]

    # Precompute smoothed transitions
    a = np.roll(forward_probabilities, shift=1, axis=1)
    a[:,0] = 0 # Set initial to 0 as there is no t-1 for the first element
    
    # Einsum over the precomputed
    numerator = np.einsum('jt,jk,kt,kt->jkt', a, transition_matrix, densities, backward_probabilities)
    denominator = numerator.sum(axis=(0,1), keepdims=True) + 1e-6 # Sum over both J and K for normalization
    v_hat = numerator / denominator

    # Return
    return delta, u_hat, v_hat
    
def estimate_transition_matrix(v_hat):
    f_ij = np.sum(v_hat, axis=2)
    f_ii = np.sum(f_ij, axis=0)
    transition_matrix = f_ij / f_ii
    return transition_matrix.T



## Maximization Functions
**Calculate Log Likelihood**

**Estimate Model Parameters**

In [8]:
def calculate_log_likelihood(N, alpha):
    # ll = np.log(np.sum(alpha[:,-1]))
    ll = np.sum(alpha[:,-1])
    return ll
def estimate_model_parameters(residuals,states):
    K, T = residuals.shape
    sum_states = np.sum(states, axis=-1)
    correlation_matrix = np.sum(np.einsum('it,jt,nt->nijt', residuals, residuals, states), axis=-1) /sum_states
    return correlation_matrix


## Uniting Functions

In [9]:
def form_transition_matrix(N):
    diagonal = 0.98
    transition_matrix = diagonal * np.eye(N) + (1-diagonal) * (np.ones((N,N)) - np.eye(N,N)) / (N - 1)
    return transition_matrix
def setup(N, data, univariate_parameters):
    initial_states = initial_state(N)
    standard_deviations = calculate_standard_deviations(data, univariate_parameters)
    residuals = calculate_residuals(data, standard_deviations)
    correlation_matrix = generate_initial_correlation_matrices(N, K)
    transition_matrix = form_transition_matrix(N)
    return initial_states, standard_deviations, residuals, correlation_matrix, transition_matrix
    
def e_step(N, K, T, standard_deviations, correlation_matrix, residuals, initial_states,transition_matrix):
    # Get the densities for the data 
    det_R, inv_R, densities = get_densities(N, K, T, standard_deviations, correlation_matrix, residuals)

    # Forward Pass
    alpha, scales = forward_pass(N, T, initial_states, densities, transition_matrix)

    # Backward Pass
    beta = backward_pass(N, T, scales, transition_matrix, densities)

    # Smoothed Probabilities
    initial_states, u_hat, v_hat = calculate_smoothed_probabilities(alpha, beta, transition_matrix, densities)

    # Estimate Transition Matrix
    estimated_transition = estimate_transition_matrix(v_hat)

    # Return
    return initial_states, densities, u_hat, v_hat, estimated_transition, alpha, beta
    
def m_step(N, K, alpha, residuals, u_hat):
    current_likelihood = calculate_log_likelihood(N, alpha)
    matrix = estimate_model_parameters(residuals,u_hat)
    correlation_matrix = np.zeros((N, K, K)) 
    for i in range(N):
        correlation_matrix[i,:,:] = cholesky_scale(matrix[i,:,:])
    
    return current_likelihood, correlation_matrix

def fit(data, univariate_parameters, num_states, max_it = 200, tol = 1e-7):
    K, T = data.shape
    N = num_states
    max_iterations = max_it
    initial_states, standard_deviations, residuals, correlation_matrix, transition_matrix = setup(N, data, univariate_parameters)
    tolerance = tol
    log_likelihood_history = np.zeros(max_iterations)
    for i in range(max_iterations): # tqdm(range(max_iterations)):
        # E step
        initial_states, densities, u_hat, v_hat, transition_matrix, alpha, beta = e_step(N, K, T, standard_deviations, correlation_matrix, residuals, initial_states,transition_matrix)

        # M step
        current_likelihood, correlation_matrix = m_step(N, K, alpha, residuals, u_hat)
        # print(current_likelihood)
        # print(correlation_matrix)
        # Track
        log_likelihood_history[i] = current_likelihood

        # Break
        if i > 1 and np.abs(log_likelihood_history[i] - log_likelihood_history[i-1]) < tolerance:
            #print(f'Model Converged at iteration:  {i}')
            break

    return correlation_matrix, transition_matrix, log_likelihood_history, u_hat

# Multithread Approach


In [ ]:
import concurrent.futures
import numpy as np
import matplotlib.pyplot as plt
# Assuming simulate_data and fit functions are properly defined elsewhere

def simulate_and_fit(iteration, params):
    try:
        # print(f"simulate_and_fit called with iteration {iteration}")     
        K, T, univariate_parameters, true_transition_matrix, cholesky = params
        # Simulate data
        data, states, variances, innovations = simulate_data(K, T, univariate_parameters, true_transition_matrix, cholesky)
        # Fit model
        correlation_matrix, transition_matrix, log_hist, u_hat = fit(data, univariate_parameters, num_states=2)
        
        # Analyze correlation matrices
        values = np.array([correlation_matrix[i][np.triu_indices(K, 1)] for i in range(N)]).flatten()
        max_value = np.max(values)
        min_value = np.min(values)
        max_rho = max_value - 0.7
        min_rho = min_value + 0.6
        
        # Analyze transition matrices
        trans_diff = np.diag(transition_matrix) - 0.99
        values = np.array([correlation_matrix[i][np.triu_indices(K, 1)] for i in range(N)]).flatten()
        return iteration, max_rho, min_rho, trans_diff
    except Exception as e:
        print(f"Exception in simulate_and_fit: {e}")
        raise  #
    
def plot_and_save(results, iteration):
    # Example plotting logic - replace with your specific analysis
    plt.figure(figsize=(10, 4))
    plt.subplot(1, 3, 1)
    plt.hist(results['max_rho'], color='skyblue', label='Max Rho') # bins=50,
    plt.title('Max Rho')
    plt.subplot(1, 3, 2)
    plt.hist(results['min_rho'], color='salmon', label='Min Rho') # bins=50,
    plt.title('Min Rho')
    plt.subplot(1, 3, 3)
    plt.hist(results['trans_diff'], color='lightgreen', label='Transition Diff') # bins=50,
    plt.title('Transition Diff')
    plt.tight_layout()
    plt.savefig(f'RSDC_test_iteration_{iteration + 1}.png')
    plt.close()

# Parameters setup as previously defined

# Parameters setup
univariate_parameters = np.array([[0.05, 0.15, 0.75], [0.1, 0.25, 0.6]])
true_correlation_matrix = np.array([[[1, 0.7], [0.7, 1]], [[1, -0.6], [-0.6, 1]]])
diagonal = 0.99
N = 2
K = 2
T = 100000
iterations = 10000  # Reduced for demonstration
plot_every = 10

true_transition_matrix = diagonal * np.eye(N) + (1-diagonal) * (np.ones((N,N)) - np.eye(N,N)) / (N - 1)
cholesky = cholesky_form(true_correlation_matrix, N, K)

# Prepare parameters for each task
tasks = [(K, T, univariate_parameters, true_transition_matrix, cholesky) for _ in range(iterations)]

# Initialize result storage


results = {'max_rho': [], 'min_rho': [], 'trans_diff': []}

with concurrent.futures.ProcessPoolExecutor() as executor:
    futures = [executor.submit(simulate_and_fit, iteration, (K, T, univariate_parameters, true_transition_matrix, cholesky)) for iteration in range(iterations)]
    # print("Futures submitted.")  # Debug print
    for future in concurrent.futures.as_completed(futures):
        # print("Processing future.")  # Debug print
        try:
            result = future.result()

            iteration, max_rho, min_rho, trans_diff = result
            print(f'Iteration: {iteration}, Output: {max_rho}{min_rho}{trans_diff[0]}{trans_diff[1]} ')
            results['max_rho'].append(max_rho)
            results['min_rho'].append(min_rho)
            results['trans_diff'].extend(trans_diff)  # Assuming trans_diff is an array

            if (iteration + 1) % plot_every == 0:
                plot_and_save(results, iteration)
                #results = {'max_rho': [], 'min_rho': [], 'trans_diff': []}

        except Exception as exc:
            print(f'Generated an exception: {exc}')


In [ ]:
import concurrent.futures
import numpy as np
import matplotlib.pyplot as plt
# Assuming simulate_data and fit functions are properly defined elsewhere

def simulate_and_fit(iteration, params):
    try:
        # print(f"simulate_and_fit called with iteration {iteration}")     
        K, T, univariate_parameters, true_transition_matrix, cholesky = params
        # Simulate data
        data, states, variances, innovations = simulate_data(K, T, univariate_parameters, true_transition_matrix, cholesky)
        # Fit model
        correlation_matrix, transition_matrix, log_hist, u_hat = fit(data, univariate_parameters, num_states=2)
        
        # Analyze correlation matrices
        values = np.array([correlation_matrix[i][np.triu_indices(K, 1)] for i in range(N)]).flatten()
        max_value = np.max(values)
        min_value = np.min(values)
        max_rho = max_value - 0.7
        min_rho = min_value + 0.6
        
        # Analyze transition matrices
        trans_diff = np.diag(transition_matrix) - 0.99
        values = np.array([correlation_matrix[i][np.triu_indices(K, 1)] for i in range(N)]).flatten()
        return iteration, max_rho, min_rho, trans_diff
    except Exception as e:
        print(f"Exception in simulate_and_fit: {e}")
        raise  #
    
def plot_and_save(resultss, iteration):
    # Example plotting logic - replace with your specific analysis
    plt.figure(figsize=(10, 4))
    plt.subplot(1, 3, 1)
    plt.hist(resultss['max_rho'], color='skyblue', label='Max Rho') # bins=50,
    plt.title('Max Rho')
    plt.subplot(1, 3, 2)
    plt.hist(resultss['min_rho'], color='salmon', label='Min Rho') # bins=50,
    plt.title('Min Rho')
    plt.subplot(1, 3, 3)
    plt.hist(resultss['trans_diff'], color='lightgreen', label='Transition Diff') # bins=50,
    plt.title('Transition Diff')
    plt.tight_layout()
    plt.savefig(f'1 mil. RSDC_test_iteration_{iteration + 1}.png')
    plt.close()

# Parameters setup as previously defined

# Parameters setup
univariate_parameters = np.array([[0.05, 0.15, 0.75], [0.1, 0.25, 0.6]])
true_correlation_matrix = np.array([[[1, 0.7], [0.7, 1]], [[1, -0.6], [-0.6, 1]]])
diagonal = 0.99
N = 2
K = 2
T = 1000000
iterations = 1000  # Reduced for demonstration
plot_every = 10

true_transition_matrix = diagonal * np.eye(N) + (1-diagonal) * (np.ones((N,N)) - np.eye(N,N)) / (N - 1)
cholesky = cholesky_form(true_correlation_matrix, N, K)

# Prepare parameters for each task
tasks = [(K, T, univariate_parameters, true_transition_matrix, cholesky) for _ in range(iterations)]

# Initialize result storage


resultss = {'max_rho': [], 'min_rho': [], 'trans_diff': []}

with concurrent.futures.ProcessPoolExecutor() as executor:
    futures = [executor.submit(simulate_and_fit, iteration, (K, T, univariate_parameters, true_transition_matrix, cholesky)) for iteration in range(iterations)]
    # print("Futures submitted.")  # Debug print
    for future in concurrent.futures.as_completed(futures):
        # print("Processing future.")  # Debug print
        try:
            result = future.result()

            iteration, max_rho, min_rho, trans_diff = result
            print(f'Iteration: {iteration}, Output: {max_rho}{min_rho}{trans_diff[0]}{trans_diff[1]} ')
            resultss['max_rho'].append(max_rho)
            resultss['min_rho'].append(min_rho)
            resultss['trans_diff'].extend(trans_diff)  # Assuming trans_diff is an array

            if (iteration + 1) % plot_every == 0:
                plot_and_save(resultss, iteration)
                #resultss = {'max_rho': [], 'min_rho': [], 'trans_diff': []}

        except Exception as exc:
            print(f'Generated an exception: {exc}')


Iteration: 3, Output: 0.0077417376073780630.006222255962125445-5.1064844858172975e-060.00010025248210354665 
Iteration: 0, Output: 0.0077417376073780630.006222255962125445-5.1064844858172975e-060.00010025248210354665 
Iteration: 1, Output: 0.0077417376073780630.006222255962125445-5.1064844858172975e-060.00010025248210354665 
Iteration: 7, Output: 0.0077417376073780630.006222255962125445-5.1064844858172975e-060.00010025248210354665 
Iteration: 2, Output: 0.0077417376073780630.006222255962125445-5.1064844858172975e-060.00010025248210354665 
Iteration: 5, Output: 0.0077417376073780630.006222255962125445-5.1064844858172975e-060.00010025248210354665 
Iteration: 4, Output: 0.0077417376073780630.006222255962125445-5.1064844858172975e-060.00010025248210354665 
Iteration: 6, Output: 0.0077417376073780630.006222255962125445-5.1064844858172975e-060.00010025248210354665 
Iteration: 9, Output: 0.010121492852930180.004319411512055282-0.000168172186039150522.3283068945723073e-05 
Iteration: 12, Outpu

In [ ]:
import time
import numpy as np

start_time = time.time()
parameters = np.array([[0.05, 0.15, 0.75],
                      [0.1, 0.25, 0.6],
                      [0.15, 0.20, 0.7]])

correlation_matrix =  np.array([[[1, 0.6, 0.4],[0.6, 1, -0.3],[0.4, -0.3, 1]],
                               [[1, -0.6, 0.2],[-0.6, 1, 0.3],[0.2, 0.3, 1]],
                               [[1, 0.9, 0.4],[0.9, 1, -0.1],[0.4, -0.1, 1]]])
diagonal = 0.99
N = 3
K = 3
transition_matrix = diagonal * np.eye(N) + (1-diagonal) * (np.ones((N,N)) - np.eye(N,N)) / (N - 1)
decomp = np.zeros((N, K, K))
for n in range(N):
    decomp[n,:,:] = choleski_form()

In [ ]:
fdsa

## Running


In [ ]:
start_time = time.time()

correlation_matrix, transition_matrix, log_hist, u_hat = fit(data, univariate_parameters, num_states=2)
# End timing
end_time = time.time()

# Calculate elapsed time
elapsed_time = end_time - start_time

# Print elapsed time with 5 digits after the decimal point
print(f"Elapsed time: {elapsed_time:.5f} seconds")


In [ ]:
transition_matrix


# Convergence Testing



In [ ]:
import matplotlib.pyplot as plt
univariate_parameters = np.array([[0.05, 0.15, 0.75],
                      [0.1, 0.25, 0.6] ])#,
                      # [0.15, 0.15, 0.8],
                      # [0.2, 0.3, 0.65]])
true_correlation_matrix = np.array([[[1, 0.7],[0.7,1]],[[1, -0.6],[-0.6, 1]]])
diagonal = 0.99
N = 2
K = 2
true_transition_matrix = diagonal * np.eye(N) + (1-diagonal) * (np.ones((N,N)) - np.eye(N,N)) / (N - 1)

cholesky = cholesky_form(true_correlation_matrix, N, K)

# Number of observations
T = 10000
iterations = 10000
plot_every = 10

max_rho = []
min_rho = []
trans_diff = []

for iteration in range(iterations):
    # Simulate data
    data, states, variances, innovations = simulate_data(K, T, univariate_parameters, true_transition_matrix, cholesky)
    
    # Fit model
    correlation_matrix, transition_matrix, log_hist = fit(data, univariate_parameters, num_states=2)

    values = np.zeros(2)
    # Analyze correlation matrices
    for i in range(N):
        values[i] = correlation_matrix[i][np.triu_indices(K, 1)]



    max = np.max(values)
    min = np.min(values)
    max_rho.append(max - 0.7)
    min_rho.append(min + 0.6)
    # Analyze transition matrices
    trans_diff.extend(np.diag(transition_matrix) - 0.99)
    
    # Plot if condition is met
    if (iteration + 1) % plot_every == 0:
        plt.figure(figsize=(10, 4))
        plt.subplot(1, 3, 1)
        plt.hist(max_rho, color='skyblue', label='Max Rho')
        plt.title('Max Rho')
        plt.subplot(1, 3, 2)
        plt.hist(min_rho, color='salmon', label='Min Rho')
        plt.title('Min Rho')
        plt.subplot(1, 3, 3)
        plt.hist(trans_diff, color='lightgreen', label='Transition Diff')
        plt.title('Transition Diff')
        plt.tight_layout()
        plt.savefig(f'RSDC test iteration {iteration}.png')
        plt.close()
# data, states, variances, innovations = simulate_data(K,T, univariate_parameters, true_transition_matrix, cholesky)

# correlation_matrix, transition_matrix, log_hist, u_hat = fit(data, univariate_parameters, num_states=2)

In [ ]:
densities.shape,np.max(densities), np.min(densities), densities

In [ ]:
np.sum(estimated_transition, axis=1)

In [ ]:
alpha, beta

In [ ]:
initial_states, u_hat, v_hat

In [ ]:
estimated_transition

In [ ]:

start_time = time.time()


standard_deviations = calculate_standard_deviations(data, parameters)
residuals = calculate_residuals(data, standard_deviations)
true_states = form_states(states, N, T)
correlation_matrix = estimate_model_parameters(res, true_states)
scaled = np.zeros((N, K, K))
for n in range(N):
    scaled[n,:,:] = cholesky_scale(correlation_matrix[n,:,:])


print(scaled)
# End timing
end_time = time.time()

# Calculate elapsed time
elapsed_time = end_time - start_time

# Print elapsed time with 5 digits after the decimal point
print(f"Elapsed time: {elapsed_time:.5f} seconds")


In [ ]:
correlation_matrix

In [ ]:
scaled

In [ ]:
true_correlation_matrix

In [ ]:
transition_matrix